In [1]:
# importing necessary libraries
import pandas as pd
import yfinance as yf
import requests
from datetime import timedelta


In [2]:
df = pd.read_csv(r'raw_data/sp_500_historical_components.csv')

# Data cleaning
In this portion of the code we will 
1) Get the list of stocks that ever existed from sp_500_historical_components.csv from https://github.com/hanshof/sp500_constituents/blob/main/sp_500_historical_components.csv
2) Use yfinance library to get historical data (2013-2020) OHLCV, PE ratio, PB ratio from the lists of stocks, indicate their presence in the stock data at any point of time on a daily level
3) Only use stocks that are listed at the start of the month to the end of the month, e.g: if a stock enters the stock market on 4 Jan 2016, we will only use the data starting from 1 feb onwards
4) Filter out stocks that newly entered the stock market before 2016, this is to ensure that we have sufficient training data for each stock

In [3]:
# Firstly, we will get the list of all stocks that ever existed in the S&P 500 from 2013 to 2020

# Convert 'date' column to datetime
df['date'] = pd.to_datetime(df['date'])

# Ensure 'tickers' is string
df['tickers'] = df['tickers'].astype(str)

# Filter date range
df = df[(df['date'] >= '2012-01-01') & (df['date'] < '2021-01-08')]

df2 = df[(df['date'] >= '2013-01-01') & (df['date'] < '2021-01-08')]#For 2013-2020 analysis only
df2 = df2.copy()


In [4]:
# Function to parse tickers from CSV string
def parse_ticker_list(s):
    if pd.isna(s): 
        return []
    s = s.strip()
    if s.startswith('"') and s.endswith('"'):
        s = s[1:-1]
    items = [t.strip() for t in s.split(',') if t.strip() != '']
    return items

# Apply parsing
df['ticker_list'] = df['tickers'].apply(parse_ticker_list)
df2['ticker_list'] = df2['tickers'].apply(parse_ticker_list)    #For 2013-2020 analysis only

# Keep only relevant columns
df = df[['date', 'ticker_list']]
df2 = df2[['date', 'ticker_list']]  #For 2013-2020 analysis only


In [5]:
def replace_fb_with_meta(ticker_list):
    return ['META' if ticker == 'FB' else ticker for ticker in ticker_list]

df['ticker_list'] = df['ticker_list'].apply(replace_fb_with_meta)
df2['ticker_list'] = df2['ticker_list'].apply(replace_fb_with_meta)   #For 2013-2020 analysis only

In [6]:
# Function to compute monthly survivors
def get_monthly_survivors(df):
    """
    Computes tickers that survived each month and those that didn't.
    
    Parameters:
        df (pd.DataFrame): DataFrame with columns 'date' (datetime) and 'ticker_list' (list of tickers)
    
    Returns:
        pd.DataFrame: Each row contains:
            - 'date': first date of the month
            - 'ticker_list': list of tickers present every day in the month
            - 'not_survived': tickers missing on any day of the month
            - 'all_tickers': total number of unique tickers observed in the month
    """
    df['year_month'] = df['date'].dt.to_period('M')
    
    # Get first date of each month
    month_starts = df.groupby('year_month')['date'].min().reset_index()
    month_starts.columns = ['year_month', 'start_date']
    
    results = []
    
    for i in range(len(month_starts) - 1):
        start = month_starts.loc[i, 'start_date']
        end = month_starts.loc[i + 1, 'start_date']

        # Filter month data
        month_df = df[(df['date'] >= start) & (df['date'] < end)]
        if month_df.empty:
            continue

        # All tickers ever observed in this month
        all_tickers = set().union(*month_df['ticker_list'])

        # Intersection across all days → surviving tickers
        surviving = set(month_df.iloc[0]['ticker_list'])
        for tickers in month_df['ticker_list']:
            surviving &= set(tickers)

        not_survived = all_tickers - surviving

        results.append({
            'date': start,
            'ticker_list': sorted(list(surviving)),
            'not_survived': sorted(list(not_survived)),
            'all_tickers': len(all_tickers)
        })

    return pd.DataFrame(results)

survivors_df = get_monthly_survivors(df)
survivors_df2 = get_monthly_survivors(df2)


In [7]:
# Function to process survivors DataFrame
def process_survivors_df(survivors_df, start_date='2012-01-01', end_date='2020-12-01'):
    """
    Processes a monthly survivors DataFrame:
    - Adds a 'num_survivors' column
    - Reindexes to include all months in the specified range
    - Forward-fills missing months
    
    Parameters:
        survivors_df (pd.DataFrame): must have columns 'date' (datetime) and 'ticker_list' (list)
        start_date (str or pd.Timestamp): first month in the reindexed range
        end_date (str or pd.Timestamp): last month in the reindexed range
    
    Returns:
        pd.DataFrame: processed DataFrame with columns:
            - 'month' (datetime)
            - 'ticker_list' (list of tickers)
            - 'not_survived' (list)
            - 'all_tickers' (int)
            - 'num_survivors' (int)
    """
    df = survivors_df.copy()
    
    # Calculate number of surviving tickers
    df['num_survivors'] = df['ticker_list'].apply(len)
    
    # Convert 'date' to month start timestamp
    df['month'] = df['date'].dt.to_period('M').dt.to_timestamp()
    df.set_index('month', inplace=True)
    
    # Reindex to full month range and forward-fill missing months
    full_month_range = pd.date_range(start=start_date, end=end_date, freq='MS')
    df = df.reindex(full_month_range)
    df.ffill(inplace=True)
    
    # Reset index and rename
    df.reset_index(inplace=True)
    df.rename(columns={'index': 'month'}, inplace=True)
    
    return df

survivors_df = process_survivors_df(survivors_df, start_date='2012-01-01', end_date='2020-12-01')
survivors_df2 = process_survivors_df(survivors_df2, start_date='2013-01-01', end_date='2020-12-01')



In [8]:
# Function to create presence matrix
def create_presence_matrix(survivors_df, date_col='date', ticker_col='ticker_list'):
    """
    Converts a monthly survivors DataFrame into a pivoted presence matrix.
    
    Parameters:
        survivors_df (pd.DataFrame): must have columns for month/date and 'ticker_list'
        date_col (str): name of the column containing the month/date
        ticker_col (str): name of the column containing the list of tickers
        
    Returns:
        pd.DataFrame: pivoted DataFrame where:
            - index = months/dates
            - columns = tickers
            - values = 1 if ticker present that month, 0 otherwise
    """
    df = survivors_df.copy()
    
    # Explode the ticker list so each row has one ticker
    exploded = df.explode(ticker_col)
    
    # Assign presence flag
    exploded['value'] = 1
    
    # Pivot the table
    pivot_df = exploded.pivot_table(
        index=date_col,
        columns=ticker_col,
        values='value',
        fill_value=0
    )
    
    # Sort rows and columns
    pivot_df = pivot_df.sort_index(axis=0).sort_index(axis=1)
    
    return pivot_df

pivot_df = create_presence_matrix(survivors_df)
pivot_df2 = create_presence_matrix(survivors_df2)


In [9]:
# Function to summarize ticker activity
def summarize_ticker_activity(pivot_df):
    """
    Summarizes the activity of each ticker from a pivoted presence matrix.
    
    Parameters:
        pivot_df (pd.DataFrame): rows = months/dates, columns = tickers, values = 1 if present, 0 if not
    
    Returns:
        pd.DataFrame: summary DataFrame with columns:
            - 'ticker': ticker symbol
            - 'first_seen': first month ticker was present
            - 'last_seen': last month ticker was present
            - 'months_active': total number of months ticker was active
    """
    summary_data = []

    for ticker in pivot_df.columns:
        series = pivot_df[ticker]
        active_months = series[series == 1]

        if not active_months.empty:
            summary_data.append({
                'ticker': ticker,
                'first_seen': active_months.index.min(),
                'last_seen': active_months.index.max(),
                'months_active': active_months.count()
            })

    summary_df = pd.DataFrame(summary_data)
    summary_df = summary_df.sort_values('ticker').reset_index(drop=True)
    
    return summary_df

summary_df = summarize_ticker_activity(pivot_df)
summary_df2 = summarize_ticker_activity(pivot_df2)

In [10]:
def compare_summaries(df_full, df_subset):
    """
    Compares two summary DataFrames and flags tickers as 'Dropped', 'New', or 'Common'.
    
    Parameters:
        df_full (pd.DataFrame): full summary (e.g., 2012-2020)
        df_subset (pd.DataFrame): subset summary (e.g., 2013-2020)
    
    Returns:
        pd.DataFrame: merged DataFrame with status and side-by-side activity info
    """
    # Merge on ticker
    comparison_df = df_full.merge(
        df_subset, 
        on='ticker', 
        how='outer', 
        suffixes=('_2012_2020', '_2013_2020')
    ).sort_values('ticker')

    # Add status column
    comparison_df['status'] = comparison_df.apply(
        lambda row: 'Dropped' if pd.notna(row['months_active_2012_2020']) and pd.isna(row['months_active_2013_2020'])
        else ('New' if pd.isna(row['months_active_2012_2020']) and pd.notna(row['months_active_2013_2020'])
        else 'Common'),
        axis=1
    )

    return comparison_df

comparison_df = compare_summaries(summary_df, summary_df2)

comparison_df[comparison_df['status'] == 'Dropped']


,ticker,first_seen_2012_2020,last_seen_2012_2020,months_active_2012_2020,first_seen_2013_2020,last_seen_2013_2020,months_active_2013_2020,status
56,ATGE,2012-01-03,2012-09-05,9,NaT,NaT,NaN,Dropped
105,CBE,2012-01-03,2012-11-06,11,NaT,NaT,NaN,Dropped
202,EP,2012-01-03,2012-04-02,4,NaT,NaT,NaN,Dropped
231,FII,2012-01-03,2012-12-03,12,NaT,NaT,NaN,Dropped
266,GR,2012-01-03,2012-06-01,6,NaT,NaT,NaN,Dropped
372,LXK,2012-01-03,2012-09-05,9,NaT,NaT,NaN,Dropped
392,MHS,2012-01-03,2012-03-01,3,NaT,NaT,NaN,Dropped
398,MMI,2012-01-03,2012-04-02,4,NaT,NaT,NaN,Dropped
511,RRD,2012-01-03,2012-11-06,11,NaT,NaT,NaN,Dropped
543,SVU,2012-01-03,2012-04-02,4,NaT,NaT,NaN,Dropped


In [11]:
# Function to get first and last day of the month
# def get_month_range(start, end):
#     start_of_month = start.to_period('M').to_timestamp()       # first day of month
#     end_of_month = end.to_period('M').to_timestamp('M')        # last day of month
#     return start_of_month, end_of_month

# Function to fetch stock data from Yahoo Finance
def get_stock_data(ticker, start_date, end_date):
    try:
        stock = yf.Ticker(ticker)
        stock_data = stock.history(start=start_date, end=end_date + pd.Timedelta(days=1))  # Add 1 day to include end date
        stock_data['ticker'] = ticker
        return stock_data
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()

# Loop through summary_df and collect stock data
stock_data_list = []
success_count = 0  # Counter for successful tickers

for idx, row in summary_df.iterrows():
    ticker = row['ticker']
    # first_seen = row['first_seen']
    # last_seen = row['last_seen']
    start_date = pd.Timestamp('2012-01-01')
    end_date = pd.Timestamp('2020-12-31')
    
    # Adjust to first and last day of the month
    # start_date, end_date = get_month_range(first_seen, last_seen)
    
    stock_data = get_stock_data(ticker, start_date , end_date)
    
    if not stock_data.empty:
        stock_data_list.append(stock_data)
        success_count += 1  # Increment if data fetched successfully

# Combine all stock data
all_stock_data = pd.concat(stock_data_list)
all_stock_data.reset_index(inplace=True)

# Print count of tickers successfully fetched
print(f"Number of tickers successfully fetched: {success_count}")

$ABC: possibly delisted; no timezone found
$ABMD: possibly delisted; no timezone found
$ADS: possibly delisted; no timezone found
$ALXN: possibly delisted; no timezone found
$ANSS: possibly delisted; no timezone found
$ANTM: possibly delisted; no timezone found
$APC: possibly delisted; no timezone found
$APOL: possibly delisted; no price data found  (1d 2012-01-01 00:00:00 -> 2021-01-01 00:00:00)
$ARG: possibly delisted; no price data found  (1d 2012-01-01 00:00:00 -> 2021-01-01 00:00:00)
$ATVI: possibly delisted; no timezone found
$AVP: possibly delisted; no timezone found
$BCR: possibly delisted; no price data found  (1d 2012-01-01 00:00:00 -> 2021-01-01 00:00:00)
$BIG: possibly delisted; no timezone found
$BLL: possibly delisted; no timezone found
$BRCM: possibly delisted; no price data found  (1d 2012-01-01 00:00:00 -> 2021-01-01 00:00:00)
$BTUUQ: possibly delisted; no price data found  (1d 2012-01-01 00:00:00 -> 2021-01-01 00:00:00)
$BXLT: possibly delisted; no price data found  (

Number of tickers successfully fetched: 525


In [12]:
all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,Capital Gains
0,2012-01-03 00:00:00-05:00,22.776927,23.507771,22.713375,23.183659,4156394,0.0,0.0,A,NaN
1,2012-01-04 00:00:00-05:00,22.973936,23.107394,22.618046,22.999355,4651845,0.0,0.0,A,NaN
2,2012-01-05 00:00:00-05:00,22.802346,23.717490,22.700664,23.514126,6842651,0.0,0.0,A,NaN
3,2012-01-06 00:00:00-05:00,23.571315,23.870008,23.393371,23.768326,4711400,0.0,0.0,A,NaN
4,2012-01-09 00:00:00-05:00,23.908135,24.416549,23.812808,24.391129,4429563,0.0,0.0,A,NaN
...,...,...,...,...,...,...,...,...,...,...
1143100,2020-12-24 00:00:00-05:00,153.424280,155.295071,153.357131,154.191788,417400,0.0,0.0,ZTS,NaN
1143101,2020-12-28 00:00:00-05:00,154.882551,156.216091,153.798445,155.793961,1522400,0.0,0.0,ZTS,NaN
1143102,2020-12-29 00:00:00-05:00,156.580664,158.393894,155.803553,156.494308,1188400,0.0,0.0,ZTS,NaN
1143103,2020-12-30 00:00:00-05:00,156.868456,158.106066,156.532681,157.597595,1009000,0.0,0.0,ZTS,NaN


In [13]:
# List of tickers to remove as they were not in S&P 500 between 2013-2020 
tickers_to_remove = [
    "ATGE", "CBE", "EP", "FII", "GR", "LXK", 
    "MHS", "MMI", "RRD", "SVU", "TIE"
]

# Remove rows where 'ticker' is in the list
all_stock_data = all_stock_data[~all_stock_data['ticker'].isin(tickers_to_remove)]


In [14]:
SP500_all_stock_data = all_stock_data.drop(columns=['Stock Splits']) # dropping since we already account for the price adjustments in 'Adj Close'


#Daily returns
SP500_all_stock_data['daily_return'] = (
    SP500_all_stock_data.groupby('ticker')['Close']
    .pct_change()
)

#Monthly returns
monthly_returns = (
    SP500_all_stock_data
    .set_index('Date')
    .groupby('ticker')['Close']
    .resample('ME')
    .ffill()             # forward-fill missing days within month
    .pct_change()        # monthly percentage change
    .reset_index(name='monthly_return')
)

SP500_all_stock_data = SP500_all_stock_data.merge(
    monthly_returns[['Date', 'ticker', 'monthly_return']],
    on=['Date', 'ticker'],
    how='left'
)

#Check index membership, if tikcer is in ticker_list for that month
SP500_all_stock_data['Date'] = SP500_all_stock_data['Date'].dt.tz_localize(None)
SP500_all_stock_data['month'] = SP500_all_stock_data['Date'].dt.to_period('M').dt.to_timestamp()


SP500_all_stock_data = SP500_all_stock_data.merge(
    survivors_df[['month', 'ticker_list']],
    on='month',
    how='left'
)

SP500_all_stock_data['membership_index'] = SP500_all_stock_data.apply(
    lambda row: row['ticker'] in row['ticker_list'],
    axis=1
)

SP500_all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,ticker,Capital Gains,daily_return,monthly_return,month,ticker_list,membership_index
0,2012-01-03,22.776927,23.507771,22.713375,23.183659,4156394,0.0,A,NaN,NaN,NaN,2012-01-01,"[A, AAPL, ABC, ABT, ACN, ADBE, ADI, ADM, ADP, ...",True
1,2012-01-04,22.973936,23.107394,22.618046,22.999355,4651845,0.0,A,NaN,-0.007950,NaN,2012-01-01,"[A, AAPL, ABC, ABT, ACN, ADBE, ADI, ADM, ADP, ...",True
2,2012-01-05,22.802346,23.717490,22.700664,23.514126,6842651,0.0,A,NaN,0.022382,NaN,2012-01-01,"[A, AAPL, ABC, ABT, ACN, ADBE, ADI, ADM, ADP, ...",True
3,2012-01-06,23.571315,23.870008,23.393371,23.768326,4711400,0.0,A,NaN,0.010811,NaN,2012-01-01,"[A, AAPL, ABC, ABT, ACN, ADBE, ADI, ADM, ADP, ...",True
4,2012-01-09,23.908135,24.416549,23.812808,24.391129,4429563,0.0,A,NaN,0.026203,NaN,2012-01-01,"[A, AAPL, ABC, ABT, ACN, ADBE, ADI, ADM, ADP, ...",True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130024,2020-12-24,153.424280,155.295071,153.357131,154.191788,417400,0.0,ZTS,NaN,0.005443,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True
1130025,2020-12-28,154.882551,156.216091,153.798445,155.793961,1522400,0.0,ZTS,NaN,0.010391,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True
1130026,2020-12-29,156.580664,158.393894,155.803553,156.494308,1188400,0.0,ZTS,NaN,0.004495,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True
1130027,2020-12-30,156.868456,158.106066,156.532681,157.597595,1009000,0.0,ZTS,NaN,0.007050,NaN,2020-12-01,"[A, AAL, AAP, AAPL, ABBV, ABC, ABMD, ABT, ACN,...",True


In [15]:
SP500_all_stock_data = SP500_all_stock_data.drop(columns=['ticker_list','month','Capital Gains'])

SP500_all_stock_data

,Date,Open,High,Low,Close,Volume,Dividends,ticker,daily_return,monthly_return,membership_index
0,2012-01-03,22.776927,23.507771,22.713375,23.183659,4156394,0.0,A,NaN,NaN,True
1,2012-01-04,22.973936,23.107394,22.618046,22.999355,4651845,0.0,A,-0.007950,NaN,True
2,2012-01-05,22.802346,23.717490,22.700664,23.514126,6842651,0.0,A,0.022382,NaN,True
3,2012-01-06,23.571315,23.870008,23.393371,23.768326,4711400,0.0,A,0.010811,NaN,True
4,2012-01-09,23.908135,24.416549,23.812808,24.391129,4429563,0.0,A,0.026203,NaN,True
...,...,...,...,...,...,...,...,...,...,...,...
1130024,2020-12-24,153.424280,155.295071,153.357131,154.191788,417400,0.0,ZTS,0.005443,NaN,True
1130025,2020-12-28,154.882551,156.216091,153.798445,155.793961,1522400,0.0,ZTS,0.010391,NaN,True
1130026,2020-12-29,156.580664,158.393894,155.803553,156.494308,1188400,0.0,ZTS,0.004495,NaN,True
1130027,2020-12-30,156.868456,158.106066,156.532681,157.597595,1009000,0.0,ZTS,0.007050,NaN,True


### Creating new features
#### Moving averages
https://www.investopedia.com/ask/answers/122414/what-are-most-common-periods-used-creating-moving-average-ma-lines.asp we will use short:20 days, medium: 50 days, long: 100 days moving averages
1) Simple moving average SMA: SMA_20, SMA_50, SMA_100
2) Exponential Moving Average EMA https://www.investopedia.com/terms/e/ema.asp#toc-formula-for-exponential-moving-average-ema: EMA_20, EMA_50, EMA_100
3) Relative Strength Index 14 days https://www.investopedia.com/terms/r/rsi.asp : RSI
4) Moving Average Convergence/Divergence indicator: https://www.investopedia.com/terms/m/macd.asp: MACD_26, MACD_12, MACD_9 

In [16]:
def sma(series: pd.Series, window: int) -> pd.Series:
    """Simple Moving Average."""
    return series.rolling(window, min_periods=window).mean()


In [17]:
def ema(series: pd.Series, span: int) -> pd.Series:
    """Exponential Moving Average."""
    return series.ewm(span=span, adjust=False, min_periods=span).mean()

In [18]:
def rsi_wilder(series: pd.Series, period: int = 14) -> pd.Series:
    """Wilder's RSI (default 14)."""
    delta = series.diff()
    gain  = delta.clip(lower=0)
    loss  = -delta.clip(upper=0)
    avg_gain = gain.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    avg_loss = loss.ewm(alpha=1/period, adjust=False, min_periods=period).mean()
    rs = avg_gain / avg_loss
    return 100 - (100 / (1 + rs))

In [19]:
def macd(series: pd.Series, fast: int = 12, slow: int = 26, signal: int = 9):
    """
    MACD parts: returns DataFrame with MACD_Line, MACD_Signal, MACD_Hist.
    """
    ema_fast = ema(series, fast)
    ema_slow = ema(series, slow)
    macd_line = ema_fast - ema_slow
    macd_signal = macd_line.ewm(span=signal, adjust=False, min_periods=signal).mean()
    macd_hist = macd_line - macd_signal
    return pd.DataFrame(
        {"MACD_Line": macd_line, "MACD_Signal": macd_signal, "MACD_Hist": macd_hist},
        index=series.index
    )

In [20]:
df = SP500_all_stock_data.copy()

# Ensure types/order; compute per-ticker
df['Date'] = pd.to_datetime(df['Date'])
df = df.drop_duplicates(subset=['ticker','Date']).sort_values(['ticker','Date'])

# Prefer adjusted close if available
price_col = 'Adj Close' if 'Adj Close' in df.columns else 'Close'

g = df.groupby('ticker', group_keys=False)

# SMA 20/50/100
for w in [20, 50, 100]:
    df[f'SMA_{w}'] = g[price_col].transform(lambda s, w=w: sma(s, w))

# EMA 20/50/100
for w in [20, 50, 100]:
    df[f'EMA_{w}'] = g[price_col].transform(lambda s, w=w: ema(s, w))

# RSI 14
df['RSI'] = g[price_col].transform(rsi_wilder)

# MACD (12,26,9)
macd_df = g[price_col].apply(macd)
df = df.join(macd_df)

# Hand back to your master frame
SP500_all_stock_data_Final = df

In [21]:
SP500_all_stock_data_Final

,Date,Open,High,Low,Close,Volume,Dividends,ticker,daily_return,monthly_return,...,SMA_20,SMA_50,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist
0,2012-01-03,22.776927,23.507771,22.713375,23.183659,4156394,0.0,A,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2012-01-04,22.973936,23.107394,22.618046,22.999355,4651845,0.0,A,-0.007950,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2012-01-05,22.802346,23.717490,22.700664,23.514126,6842651,0.0,A,0.022382,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2012-01-06,23.571315,23.870008,23.393371,23.768326,4711400,0.0,A,0.010811,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2012-01-09,23.908135,24.416549,23.812808,24.391129,4429563,0.0,A,0.026203,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130024,2020-12-24,153.424280,155.295071,153.357131,154.191788,417400,0.0,ZTS,0.005443,NaN,...,153.626691,155.672894,154.709186,154.477103,154.663487,151.699312,48.711837,-0.298370,-0.514777,0.216406
1130025,2020-12-28,154.882551,156.216091,153.798445,155.793961,1522400,0.0,ZTS,0.010391,NaN,...,153.670824,155.665291,154.755881,154.602518,154.707819,151.780395,52.813413,-0.160573,-0.443936,0.283363
1130026,2020-12-29,156.580664,158.393894,155.803553,156.494308,1188400,0.0,ZTS,0.004495,NaN,...,153.802261,155.660965,154.774989,154.782688,154.777877,151.873739,54.525367,0.005087,-0.354131,0.359218
1130027,2020-12-30,156.868456,158.106066,156.532681,157.597595,1009000,0.0,ZTS,0.007050,NaN,...,153.941851,155.746925,154.828605,155.050775,154.888454,151.987083,57.162070,0.222830,-0.238739,0.461569


In [22]:
# 2012-05-23 is the 100th trading day from 2012-01-01
final_cleaned_data = SP500_all_stock_data_Final[SP500_all_stock_data_Final['Date'] > '2012-05-23']

final_cleaned_data


,Date,Open,High,Low,Close,Volume,Dividends,ticker,daily_return,monthly_return,...,SMA_20,SMA_50,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist
99,2012-05-24,26.013021,26.210476,25.764612,26.127672,3591182,0.0,A,0.006132,NaN,...,25.924482,26.995301,26.976644,25.890111,26.463516,26.338783,50.437411,-0.354859,-0.473274,0.118415
100,2012-05-25,26.127669,26.407927,25.981171,26.121300,2633552,0.0,A,-0.000244,NaN,...,25.875118,26.934831,27.006021,25.912129,26.450095,26.334477,50.374728,-0.294990,-0.437617,0.142627
101,2012-05-29,26.414300,26.911120,26.388822,26.866535,5696570,0.0,A,0.028530,NaN,...,25.875118,26.892316,27.044692,26.003024,26.466426,26.345012,57.090821,-0.185273,-0.387149,0.201875
102,2012-05-30,26.522576,26.522576,25.987538,26.337860,4297871,0.0,A,-0.019678,NaN,...,25.823525,26.842533,27.072930,26.034914,26.461385,26.344871,51.741119,-0.139375,-0.337594,0.198219
103,2012-05-31,26.407929,26.509842,25.649958,25.898369,5887258,0.0,A,-0.016687,-0.036036,...,25.747728,26.790696,27.094230,26.021909,26.439306,26.336029,47.736510,-0.136885,-0.297452,0.160567
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130024,2020-12-24,153.424280,155.295071,153.357131,154.191788,417400,0.0,ZTS,0.005443,NaN,...,153.626691,155.672894,154.709186,154.477103,154.663487,151.699312,48.711837,-0.298370,-0.514777,0.216406
1130025,2020-12-28,154.882551,156.216091,153.798445,155.793961,1522400,0.0,ZTS,0.010391,NaN,...,153.670824,155.665291,154.755881,154.602518,154.707819,151.780395,52.813413,-0.160573,-0.443936,0.283363
1130026,2020-12-29,156.580664,158.393894,155.803553,156.494308,1188400,0.0,ZTS,0.004495,NaN,...,153.802261,155.660965,154.774989,154.782688,154.777877,151.873739,54.525367,0.005087,-0.354131,0.359218
1130027,2020-12-30,156.868456,158.106066,156.532681,157.597595,1009000,0.0,ZTS,0.007050,NaN,...,153.941851,155.746925,154.828605,155.050775,154.888454,151.987083,57.162070,0.222830,-0.238739,0.461569


In [25]:
#To check
pd.set_option('display.max_columns', None)  # Show all columns
final_cleaned_data[
    (final_cleaned_data['ticker'] == 'TSLA') & 
    (final_cleaned_data['membership_index'] == True)
]

,Date,Open,High,Low,Close,Volume,Dividends,ticker,daily_return,monthly_return,membership_index,SMA_20,SMA_50,SMA_100,EMA_20,EMA_50,EMA_100,RSI,MACD_Line,MACD_Signal,MACD_Hist
1002807,2020-12-01,199.196671,199.283340,190.683334,194.919998,120310500,0.0,TSLA,0.030233,NaN,True,158.274999,147.764132,133.619953,165.612912,150.088915,131.986521,74.653316,13.806472,8.913852,4.892620
1002808,2020-12-02,185.479996,190.513336,180.403336,189.606674,143327100,0.0,TSLA,-0.027259,NaN,True,160.690333,148.728066,134.517980,167.898032,151.638631,133.127514,69.365709,13.950904,9.921262,4.029642
1002809,2020-12-03,196.673340,199.656662,194.143326,197.793335,127656000,0.0,TSLA,0.043177,NaN,True,163.563667,150.148199,135.484713,170.745204,153.448619,134.408025,72.587411,14.558145,10.848639,3.709506
1002810,2020-12-04,197.003326,199.679993,195.166672,199.679993,88203900,0.0,TSLA,0.009539,NaN,True,166.246166,151.556532,136.450839,173.500898,155.261614,135.700540,73.284693,15.018501,11.682611,3.335890
1002811,2020-12-07,201.639999,216.263336,201.016663,213.919998,168929100,0.0,TSLA,0.071314,NaN,True,169.776333,153.119332,137.589613,177.350336,157.561943,137.249440,77.861901,16.343983,12.614886,3.729097
1002812,2020-12-08,208.503326,217.093338,206.166672,216.626663,192795000,0.0,TSLA,0.012653,NaN,True,173.586666,154.643866,138.755319,181.090939,159.878207,138.821266,78.612001,17.412126,13.574334,3.837792
1002813,2020-12-09,217.896667,218.106674,196.000000,201.493332,213873600,0.0,TSLA,-0.069859,NaN,True,176.821999,155.879932,139.674919,183.034024,161.510172,140.062297,65.291498,16.843344,14.228136,2.615208
1002814,2020-12-10,191.456665,209.250000,188.779999,209.023331,201249600,0.0,TSLA,0.037371,NaN,True,180.320999,157.200332,140.719579,185.509196,163.373434,141.427862,68.180636,16.806454,14.743799,2.062655
1002815,2020-12-11,205.003326,208.000000,198.933334,203.330002,139425000,0.0,TSLA,-0.027238,NaN,True,183.624833,158.279199,141.691326,187.206415,164.940358,142.653647,63.852822,16.131856,15.021411,1.110446
1002816,2020-12-14,206.333328,214.250000,203.399994,213.276672,156121800,0.0,TSLA,0.048919,NaN,True,187.480333,159.777466,142.815379,189.689297,166.835900,144.052123,67.709223,16.212953,15.259719,0.953234


In [24]:
# Columns to check
sma_cols = ['SMA_20', 'SMA_50', 'SMA_100']

# Calculate % of NaNs per ticker per SMA column
nan_percent_all_smas = final_cleaned_data.groupby('ticker')[sma_cols] \
    .apply(lambda x: x.isna().mean() * 100)

# Sort by SMA_20 NaN percentage descending (optional)
nan_percent_all_smas = nan_percent_all_smas.sort_values(by='SMA_20', ascending=False)

# Print the NaN % table
print("NaN percentage per ticker:")
print(nan_percent_all_smas)

# Calculate and print average NaN % per column
average_nan_percent = nan_percent_all_smas.mean()
print("\nAverage NaN percentage per SMA column:")
print(average_nan_percent)

# Average % missing across the SMA columns for each ticker
average_nan_per_ticker = nan_percent_all_smas.mean(axis=1)

count = average_nan_per_ticker[average_nan_per_ticker == 0].count()
# Show result
print(count)



NaN percentage per ticker:
            SMA_20      SMA_50     SMA_100
ticker                                    
PETM    100.000000  100.000000  100.000000
VNT      27.536232   71.014493  100.000000
CARR      9.500000   24.500000   49.500000
OTIS      9.500000   24.500000   49.500000
TE        7.692308   19.838057   40.080972
...            ...         ...         ...
YUM       0.000000    0.000000    0.000000
ZBH       0.000000    0.000000    0.000000
ZBRA      0.000000    0.000000    0.000000
ZION      0.000000    0.000000    0.000000
AAPL      0.000000    0.000000    0.000000

[517 rows x 3 columns]

Average NaN percentage per SMA column:
SMA_20     0.393375
SMA_50     0.712336
SMA_100    1.161477
dtype: float64
475
